In [1]:
from sklearn.datasets import fetch_20newsgroups

from elasticsearch import Elasticsearch, RequestError
from tqdm import tqdm

In [2]:
es = Elasticsearch()

# 20NG

In [3]:
news_train = fetch_20newsgroups(
    data_home='../data/20newsgroups/',
    subset='train', remove=('headers', 'footers', 'quotes'),
    shuffle=True, random_state=666)

In [5]:
for index, data in tqdm(enumerate(news_train.data)):
    if index > 500:
        break
    doc = {'doc_id': index,
           'doc_text': data}
    try:
        es.index(index='20ng', doc_type='doc', body=doc)
    except RequestError:
        pass


0it [00:00, ?it/s]
1it [00:00,  2.83it/s]
3it [00:00,  5.92it/s]
5it [00:00,  8.02it/s]
7it [00:00,  9.53it/s]
9it [00:00, 10.24it/s]
11it [00:01, 10.91it/s]
13it [00:01, 11.57it/s]
15it [00:01, 12.11it/s]
17it [00:01, 12.54it/s]
19it [00:01, 12.92it/s]
21it [00:01, 13.17it/s]
23it [00:01, 13.32it/s]
25it [00:01, 13.53it/s]
27it [00:01, 13.85it/s]
29it [00:02, 14.14it/s]
31it [00:02, 14.36it/s]
33it [00:02, 14.40it/s]
35it [00:02, 14.61it/s]
37it [00:02, 14.51it/s]
39it [00:02, 14.67it/s]
41it [00:02, 14.73it/s]
43it [00:02, 14.61it/s]
45it [00:03, 14.60it/s]
47it [00:03, 14.57it/s]
49it [00:03, 14.57it/s]
51it [00:03, 14.56it/s]
53it [00:03, 14.59it/s]
55it [00:03, 14.64it/s]
57it [00:03, 14.64it/s]
59it [00:04, 14.01it/s]
61it [00:04, 14.02it/s]
63it [00:04, 14.01it/s]
65it [00:04, 14.07it/s]
67it [00:04, 14.05it/s]
69it [00:04, 14.05it/s]
71it [00:05, 14.06it/s]
73it [00:05, 14.12it/s]
75it [00:05, 14.12it/s]
77it [00:05, 14.12it/s]
79it [00:05, 14.09it/s]
81it [00:05, 14.05it/s]
8

# DUC

In [6]:
def get_docs(path):
    import os
    from bs4 import BeautifulSoup
    gold_path = path + "Summaries"
    
    ret_dict = {}
    
    doc_files = os.listdir(path)
    gold_files = os.listdir(gold_path)

    for gold_file in gold_files:
        file_name =  gold_file.split('.')[0].upper()
        if file_name not in doc_files:
            print('no file for %s' % gold_file)
            continue

        sum_path = gold_path + '/' + gold_file

        with open(sum_path) as sum_file:                
            summary, doc = sum_file.read().strip().split('Introduction:')
            summary = summary.split('Abstract:')[1]
            
            ret_dict[file_name] = [doc.strip(), summary.strip()]
    return ret_dict

In [7]:
docs = get_docs('../data/DUC/')

no file for ap890325-0143.txt
no file for ap900928-0054.txt


In [8]:
for key, val in tqdm(docs.items()):
    es_inp = {
        'doc_id' : key,
        'doc_text' : val[0],
        'gold_summary' : val[1]
    }
    es.index(index='duc', doc_type='doc', body=es_inp)


 23%|██▎       | 70/301 [00:05<00:18, 12.79it/s]
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/cyclops/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/cyclops/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/home/cyclops/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 301/301 [00:22<00:00, 13.52it/s]
